In [1]:
!pip install datasets transformers[sentencepiece]

  Using cached datasets-2.7.1-py3-none-any.whl (451 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 19.0 MB/s eta 0:00:0000:0100:01
  Using cached xxhash-3.1.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached huggingface_hub-0.11.1-py3-none-any.whl (182 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached aiohttp-3.8.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached pyarrow-10.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.9 MB)
  Using cached multiprocess-0.70.14-py39-none-any.whl (132 kB)
  Using cached tokenizers-0.13.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)
  Using cached filelock-3.8.2-py3-none-any.whl (10 kB)
  Using cached regex-2022.10.31-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (769 kB)
  Using cached sentencepiece-0.1.97-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached async_timeout-4.

In [2]:
from datasets import load_dataset
cnn_dataset = load_dataset("ccdv/cnn_dailymail", "3.0.0")
cnn_dataset

Found cached dataset cnn_dailymail (/home/jovyan/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [3]:
def show_samples(dataset, num_samples = 3, seed = 42):
    sample = dataset["train"].shuffle(seed = seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Highlights: {example['highlights']}'")
        print(f"\n'>> Article: {example['article']}'")
        
show_samples(cnn_dataset)

Loading cached shuffled indices for dataset at /home/jovyan/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f/cache-094b41cff0477abb.arrow



'>> Highlights: Brittany Ferrell, nursing student, was arrested with 12 people on Thursday .
They were calling on police take responsibility for Michael Brown's death .
Ms Ferrell tweeted as she was arrested, piled in a small wagon with 7 others .
They were accused of 'noise disruption', put in orange jumpsuits and cuffed .
Officers now being investigated, lawyers claim they 'overstretched powers''

'>> Article: A protester in Ferguson was arrested during a demonstration on Thursday night - and live-tweeted her entire experience. Brittany Ferrell, a nursing student at the University of Missouri-Saint Louis, was one of 13 people detained by officers in the conflicted Missouri city for 'noise disruption'. The detention has sparked an investigation by the American Civil Liberties Union as lawyers accuse officers of overstretching their powers. Scroll down for video . Arrested: This is Brittany Ferrell, the nursing student and protester who live-tweeted her arrest in Ferguson . Tweeting i

In [4]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/opt/conda/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
max_input_length = 512
max_target_length = 30

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["article"],
        max_length = max_input_length,
        truncation = True,
    )
    labels = tokenizer(
        examples["highlights"], max_length = max_target_length, truncation = True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [6]:
tokenized_datasets = cnn_dataset.map(preprocess_function, batched = True)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f/cache-b2308ecedf4c01b6.arrow


  0%|          | 0/14 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f/cache-1306a86d8e204937.arrow


In [7]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
import evaluate
rouge_score = evaluate.load("rouge")

In [10]:
!pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
from nltk.tokenize import sent_tokenize

def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [13]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [14]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["article"]]
    return metric.compute(predictions = summaries, references = dataset["highlights"])

In [15]:
import pandas as pd

score = evaluate_baseline(cnn_dataset["validation"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 39.49, 'rouge2': 17.56, 'rougeL': 24.96, 'rougeLsum': 36.12}

In [16]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [17]:
!pip install huggingface_hub

In [18]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8

logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

In [20]:
args = Seq2SeqTrainingArguments(
    output_dir = f"{model_name}-finetuned-cnn-dailymail",
    evaluation_strategy = "epoch",
    learning_rate=5.6e-5,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay = 0.01,
    save_total_limit = 2,
    num_train_epochs = num_train_epochs,
    predict_with_generate = True,
    logging_steps=logging_steps,
    push_to_hub = True,
    resume_from_checkpoint=True,
)

In [21]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = rouge_score.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)
    result = {key: value * 100 for key, value in result.items()}
    
    return {k: round(v, 4) for k, v in result.items()}

In [22]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [23]:
tokenized_datasets = tokenized_datasets.remove_columns(
    cnn_dataset["train"].column_names
)

In [24]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  1385,    277,    263,  ...,   1866,    282,      1],
        [   274,  57063,    271,  ..., 111875,    259,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[ 55984,    321,  21421,    267,  20820,    259, 120514,    345,    288,
            287,   2672,    304,    287,  39114,    261,    313,   6975,    272,
            277,    270,   3076,   2606,    288,   1689,   5123,    311,  20820,
           6559,    263,      1],
        [ 26764,    348,  74072,  73533,    259,  18211,  17088,    304,   4836,
            259, 147758,    259,    260,    259, 111235,    263,    259,  96037,
            288,    419,    329,   2251,    282,   5686,    276,    259,  98158,
            259,    260,      1]]), 'decoder_input_ids': tensor([[     0,  55984,    321,  21421,    267,  20820,    259, 120514,    345,
            288,    287,   2672,    304,    287,  39114,    261,    313,   6975,
            272,   

In [25]:
import transformers
transformers.logging.set_verbosity_info()

In [28]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
)

/home/jovyan/Summarisation/mt5-small-finetuned-cnn-dailymail is already a clone of https://huggingface.co/MadMarx37/mt5-small-finetuned-cnn-dailymail. Make sure you pull the latest changes with `repo.git_pull()`.


In [29]:
trainer.train("./mt5-small-finetuned-cnn-dailymail/checkpoint-27000")

Loading model from ./mt5-small-finetuned-cnn-dailymail/checkpoint-27000.
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 287113
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 71784
  Number of trainable parameters = 300176768
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 3
  Continuing training from global step 27000
  Will skip the first 3 epochs then the first 81 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command,

  0%|          | 0/81 [00:00<?, ?it/s]

/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
4,2.161500,1.763996,32.678800,16.940000,28.994000,30.688300
5,2.161500,1.745028,32.812900,17.048000,29.078800,30.810600
6,2.161500,1.737945,32.707400,16.964100,28.974500,30.704300
7,2.161500,1.731707,32.769200,17.011600,29.039500,30.768500
8,2.088600,1.729406,32.835200,17.063300,29.088800,30.822600


Saving model checkpoint to mt5-small-finetuned-cnn-dailymail/checkpoint-27500
Configuration saved in mt5-small-finetuned-cnn-dailymail/checkpoint-27500/config.json
Model weights saved in mt5-small-finetuned-cnn-dailymail/checkpoint-27500/pytorch_model.bin
tokenizer config file saved in mt5-small-finetuned-cnn-dailymail/checkpoint-27500/tokenizer_config.json
Special tokens file saved in mt5-small-finetuned-cnn-dailymail/checkpoint-27500/special_tokens_map.json
Copy vocab file to mt5-small-finetuned-cnn-dailymail/checkpoint-27500/spiece.model
tokenizer config file saved in mt5-small-finetuned-cnn-dailymail/tokenizer_config.json
Special tokens file saved in mt5-small-finetuned-cnn-dailymail/special_tokens_map.json
Copy vocab file to mt5-small-finetuned-cnn-dailymail/spiece.model
Deleting older checkpoint [mt5-small-finetuned-cnn-dailymail/checkpoint-26000] due to args.save_total_limit
Deleting older checkpoint [mt5-small-finetuned-cnn-dailymail/checkpoint-26500] due to args.save_total_lim

TrainOutput(global_step=71784, training_loss=1.3120681161294876, metrics={'train_runtime': 38405.036, 'train_samples_per_second': 59.807, 'train_steps_per_second': 1.869, 'total_flos': 1.2144872799810355e+18, 'train_loss': 1.3120681161294876, 'epoch': 8.0})

In [30]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 13368
  Batch size = 32


{'eval_loss': 1.7294056415557861,
 'eval_rouge1': 32.8352,
 'eval_rouge2': 17.0633,
 'eval_rougeL': 29.0888,
 'eval_rougeLsum': 30.8226,
 'eval_runtime': 222.5164,
 'eval_samples_per_second': 60.076,
 'eval_steps_per_second': 1.879,
 'epoch': 8.0}

In [31]:
trainer.push_to_hub()

Saving model checkpoint to mt5-small-finetuned-cnn-dailymail
Configuration saved in mt5-small-finetuned-cnn-dailymail/config.json
Model weights saved in mt5-small-finetuned-cnn-dailymail/pytorch_model.bin
tokenizer config file saved in mt5-small-finetuned-cnn-dailymail/tokenizer_config.json
Special tokens file saved in mt5-small-finetuned-cnn-dailymail/special_tokens_map.json
Copy vocab file to mt5-small-finetuned-cnn-dailymail/spiece.model


Upload file pytorch_model.bin:   0%|          | 32.0k/1.12G [00:00<?, ?B/s]

Upload file runs/Dec03_02-12-52_jupyter-mkac283/events.out.tfevents.1670084941.jupyter-mkac283.50273.4: 100%|#…

Upload file runs/Dec03_02-12-52_jupyter-mkac283/events.out.tfevents.1670033739.jupyter-mkac283.50273.2: 100%|#…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/MadMarx37/mt5-small-finetuned-cnn-dailymail
   f5ab95e..6c8755d  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/MadMarx37/mt5-small-finetuned-cnn-dailymail
   f5ab95e..6c8755d  main -> main

To https://huggingface.co/MadMarx37/mt5-small-finetuned-cnn-dailymail
   6c8755d..dac9127  main -> main

   6c8755d..dac9127  main -> main



'https://huggingface.co/MadMarx37/mt5-small-finetuned-cnn-dailymail/commit/6c8755d290d2131ff02d40404ec6c263b3042eec'